# 1. Importing Modules

In [1]:
import numpy as np
import pandas as pd
import re
import random
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import os

In [2]:
# Checking the current working directory
os.getcwd()

'D:\\University at Buffalo\\Sem 1\\CSE 587 Data Intensive Computing\\CSE 587 PROJECT LAPTRACK'

# 2. Data Collection

## Flipkart

1. For Flipkart, first we will collect the URLs of product
2. The for each product page, we will scrape information from that page

In [3]:
# Defining Request Headers to scrape the data
request_header = {
    'Content-Type': 'text/html; charset=UTF-8',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/119.0',
    'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://www.flipkart.com/',
    'Origin': 'https://www.flipkart.com',
    'Accept-Language': 'en-US,en;q=0.9'
}

In [4]:
# Storing the URL as f string
page = 1
URL = f"https://www.flipkart.com/search?q=laptop&sid=6bo%2Cb5g&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_2_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_2_na_na_na&as-pos=2&as-type=RECENT&suggestionId=laptop%7CLaptops&requestId=ad586044-bac6-4f4f-93c7-456f2161e745&as-backfill=on&page={page}"

Flipkart referencce URL: https://www.flipkart.com/search?q=laptop&sid=6bo%2Cb5g&as=on&as-show=on&otracker=AS_QueryStore_OrganicAutoSuggest_2_2_na_na_na&otracker1=AS_QueryStore_OrganicAutoSuggest_2_2_na_na_na&as-pos=2&as-type=RECENT&suggestionId=laptop%7CLaptops&requestId=ad586044-bac6-4f4f-93c7-456f2161e745&as-backfill=on&page=2

In [5]:
# Scraping Code

total_pages = 88 # Total number of pages being scraped
i = 1 # Counter to self verify the pages being scraped successfully
raw_text = [] # List to store all the raw html code

# Loop to iterate over all the pages by changing the f-string URL
for page in range (1, total_pages+1):

    # Fetching the data from URL based on the above request headers
    response = requests.get(URL, headers=request_header)

    # Random number to be used as time delay in order to make the script behaviour more human like
    delay = random.randint(5,10)
    print("Time Delay:",delay,end=" seconds    : ")

    # While Loop: covers the edge case wherein the first attempt to fetch the data failed, 
    # by continuously requesting the data at irregular time intervals in order to mimic human behavior
    while response.status_code!=200:
        time.sleep(delay)
        response = requests.get(URL,headers=request_header)

    # Confirmation Message of Successful Scrape
    print("Page",i," status:",response)

    # Incrementing Page Counter
    i+=1  

    # Appending the raw HTML code in the list
    raw_text.append(response.text)

    # A random delay before requesting the data from next page
    time.sleep(delay)

Time Delay: 5 seconds    : Page 1  status: <Response [200]>
Time Delay: 5 seconds    : Page 2  status: <Response [200]>
Time Delay: 10 seconds    : Page 3  status: <Response [200]>
Time Delay: 10 seconds    : Page 4  status: <Response [200]>
Time Delay: 6 seconds    : Page 5  status: <Response [200]>
Time Delay: 7 seconds    : Page 6  status: <Response [200]>
Time Delay: 5 seconds    : Page 7  status: <Response [200]>
Time Delay: 9 seconds    : Page 8  status: <Response [200]>
Time Delay: 6 seconds    : Page 9  status: <Response [200]>
Time Delay: 8 seconds    : Page 10  status: <Response [200]>
Time Delay: 8 seconds    : Page 11  status: <Response [200]>
Time Delay: 8 seconds    : Page 12  status: <Response [200]>
Time Delay: 8 seconds    : Page 13  status: <Response [200]>
Time Delay: 6 seconds    : Page 14  status: <Response [200]>
Time Delay: 6 seconds    : Page 15  status: <Response [200]>
Time Delay: 10 seconds    : Page 16  status: <Response [200]>
Time Delay: 5 seconds    : Pag

In [6]:
# Converting the list to Data Frame
df = pd.DataFrame(raw_text,columns=["Raw Data"])

# Printing a sample to ensure correct data format
df.head()

,Raw Data
0,"<!doctype html><html lang=""en""><head><link hre..."
1,"<!doctype html><html lang=""en""><head><link hre..."
2,"<!doctype html><html lang=""en""><head><link hre..."
3,"<!doctype html><html lang=""en""><head><link hre..."
4,"<!doctype html><html lang=""en""><head><link hre..."


In [7]:
# Dataframe has been created successfully and can now be saved in a CSV file
df.to_csv(r"new data\flipkart\rawPage.csv")

## Scraping laptop - Flipkart

In [8]:
# Loading the raw page CSV
df = pd.read_csv(r"new data\flipkart\rawPage.csv")
df.head()

,Unnamed: 0,Raw Data
0,0,"<!doctype html><html lang=""en""><head><link hre..."
1,1,"<!doctype html><html lang=""en""><head><link hre..."
2,2,"<!doctype html><html lang=""en""><head><link hre..."
3,3,"<!doctype html><html lang=""en""><head><link hre..."
4,4,"<!doctype html><html lang=""en""><head><link hre..."


In [9]:
pages = []
for i,j in df.iterrows():
    pages.append(j["Raw Data"])

In [47]:
# Iterating over all pages and for each page filtering out the product URLs
product_url = []
for page in pages:
    soup = BeautifulSoup(page, 'html.parser')
    
    product_links = soup.find_all('a', class_='sku-title')
    
    # Extract the href attributes from the found anchor tags
    laptop_url = [link.get('href') for link in product_links if link.get('href')]
    
    # Best Buy URLs are relative, so prepend with the base URL
    laptop_url = ['https://www.bestbuy.com' + url if url.startswith('/site') else url for url in laptop_url]
    
    product_url.extend(laptop_url)
    print(product_url)
    break

['https://www.bestbuy.com/site/asus-proart-p16-16-4k-touch-screen-laptop-copilot-pc-amd-ryzen-ai-9-hx-370-32gb-memory-rtx-4060-1tb-ssd-nano-black/6584438.p?skuId=6584438', 'https://www.bestbuy.com/site/dell-xps-16-16-3-fhd-laptop-intel-core-ultra-7-evo-edition-16gb-memory-nvidia-geforce-rtx-4050-1tb-ssd-platinum/6575148.p?skuId=6575148', 'https://www.bestbuy.com/site/hp-envy-16-wide-ultra-xga-touch-screen-laptop-intel-core-i7-16gb-memory-nvidia-geforce-rtx-4060-1tb-ssd-natural-silver/6572058.p?skuId=6572058', 'https://www.bestbuy.com/site/lenovo-yoga-pro-9i-16-3-2k-touchscreen-laptop-intel-core-ultra-9-185h-with-32gb-memory-nvidia-geforce-rtx-4050-1tb-ssd-luna-grey/6571367.p?skuId=6571367']


25


## Best Buy

In [32]:
headers = {'User-Agent': 'Mozilla/5.0'}

In [33]:
# Storing the URL as f string
page = 1
URL = f"https://www.bestbuy.com/site/searchpage.jsp?cp={page}&id=pcat17071&st=laptop"

Reference URL:https://www.bestbuy.com/site/searchpage.jsp?cp=3&id=pcat17071&st=laptop

In [34]:
# Scraping Code

total_pages = 71 # Total number of pages being scraped
i = 1 # Counter to self verify the pages being scraped successfully
raw_text = [] # List to store all the raw html code

# Loop to iterate over all the pages by changing the f-string URL
for page in range (1, total_pages+1):

    # Fetching the data from URL based on the above request headers
    response = requests.get(URL, headers=request_header)

    # Random number to be used as time delay in order to make the script behaviour more human like
    delay = random.randint(5,10)
    print("Time Delay:",delay,end=" seconds    : ")

    # While Loop: covers the edge case wherein the first attempt to fetch the data failed, 
    # by continuously requesting the data at irregular time intervals in order to mimic human behavior
    while response.status_code!=200:
        time.sleep(delay)
        response = requests.get(URL,headers=request_header)

    # Confirmation Message of Successful Scrape
    print("Page",i," status:",response)

    # Incrementing Page Counter
    i+=1  

    # Appending the raw HTML code in the list
    raw_text.append(response.text)

    # A random delay before requesting the data from next page
    time.sleep(delay)

Time Delay: 8 seconds    : Page 1  status: <Response [200]>
Time Delay: 6 seconds    : Page 2  status: <Response [200]>
Time Delay: 10 seconds    : Page 3  status: <Response [200]>
Time Delay: 10 seconds    : Page 4  status: <Response [200]>
Time Delay: 5 seconds    : Page 5  status: <Response [200]>
Time Delay: 8 seconds    : Page 6  status: <Response [200]>
Time Delay: 10 seconds    : Page 7  status: <Response [200]>
Time Delay: 8 seconds    : Page 8  status: <Response [200]>
Time Delay: 7 seconds    : Page 9  status: <Response [200]>
Time Delay: 6 seconds    : Page 10  status: <Response [200]>
Time Delay: 9 seconds    : Page 11  status: <Response [200]>
Time Delay: 5 seconds    : Page 12  status: <Response [200]>
Time Delay: 9 seconds    : Page 13  status: <Response [200]>
Time Delay: 5 seconds    : Page 14  status: <Response [200]>
Time Delay: 8 seconds    : Page 15  status: <Response [200]>
Time Delay: 5 seconds    : Page 16  status: <Response [200]>
Time Delay: 9 seconds    : Pag

In [35]:
# Converting the list to Data Frame
df = pd.DataFrame(raw_text,columns=["Raw Data"])

# Printing a sample to ensure correct data format
df.head()

,Raw Data
0,\n \n ...
1,\n \n ...
2,\n \n ...
3,\n \n ...
4,\n \n ...


In [36]:
# Dataframe has been created successfully and can now be saved in a CSV file
df.to_csv(r"new data\bestBuy\rawPageBestBuy.csv")

Scraping Laptops: BestBuy

In [37]:
# Loading the raw page CSV
df = pd.read_csv(r"new data\bestBuy\rawPageBestBuy.csv")
df.head()

,Unnamed: 0,Raw Data
0,0,\n \n ...
1,1,\n \n ...
2,2,\n \n ...
3,3,\n \n ...
4,4,\n \n ...


In [38]:
pages = []
for i,j in df.iterrows():
    pages.append(j["Raw Data"])

In [49]:
# Iterating over all pages and for each page filtering out the product URLs
product_url = []
for page in pages:
    soup = BeautifulSoup(page, 'html.parser')
    
    # Update the class name to reflect the Best Buy HTML structure
    product_links = soup.find_all('a', class_='sku-title')
    
    # Extract the href attributes from the found anchor tags
    laptop_url = [link.get('href') for link in product_links if link.get('href')]
    
    # Best Buy URLs are relative, so prepend with the base URL
    laptop_url = ['https://www.bestbuy.com' + url if url.startswith('/site') else url for url in laptop_url]
    
    product_url.extend(laptop_url)

In [50]:
len(product_url)

284